In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd
import torch

In [ ]:
reviews = pd.read_csv('../Datasets/IMDB Dataset.csv')
reviews = reviews.head(500)["review"]
reviews

0      One of the other reviewers has mentioned that ...
1      A wonderful little production. <br /><br />The...
2      I thought this was a wonderful way to spend ti...
3      Basically there's a family where a little boy ...
4      Petter Mattei's "Love in the Time of Money" is...
                             ...                        
495    "American Nightmare" is officially tied, in my...
496    First off, I have to say that I loved the book...
497    This movie was extremely boring. I only laughe...
498    I was disgusted by this movie. No it wasn't be...
499    Such a joyous world has been created for us in...
Name: review, Length: 500, dtype: object

In [18]:
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased").to("cuda")

results = []
for review in reviews:
    # Tokenize with truncation and padding to max length
    tokenized_review = tokenizer(review, return_tensors="pt", truncation=True, padding="max_length", max_length=512).to("cuda")

    # Get the model's output (logits)
    with torch.no_grad():
        outputs = model(**tokenized_review)
    
    # Convert logits to probabilities
    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # Get the predicted label and score
    score = probabilities.max().item()
    label = "POSITIVE" if torch.argmax(probabilities).item() == 1 else "NEGATIVE"
    
    # Append the result
    results.append({"label": label, "score": score})

# Create DataFrame
sentiment_df = pd.DataFrame(results)
sentiment_df


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


,label,score
0,POSITIVE,0.508178
1,POSITIVE,0.521151
2,POSITIVE,0.528036
3,POSITIVE,0.517413
4,POSITIVE,0.520384
...,...,...
495,POSITIVE,0.528022
496,POSITIVE,0.512645
497,POSITIVE,0.524352
498,POSITIVE,0.503319
